In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

from cc_hardware.drivers.spads.pkl import PklSPADSensorConfig, PklSPADSensor

In [ ]:
pkl_path = Path("../../../logs/20240922_192338/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/10-38-07/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/11-52-34/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/11-54-54/data.pkl")
assert pkl_path.exists()

sensor = PklSPADSensor(PklSPADSensorConfig(pkl_path=pkl_path, merge=False, loop=True, key="histograms", resolution=(8, 8)))
height, width = sensor.resolution
print(f"Resolution: {height} x {width}")

index = 10

In [ ]:
histograms = sensor.accumulate(index=index).reshape(height, width, -1)

ylim = None # 1e4
min_bin = 0
max_bin = min(histograms.shape[-1] - 1, 60)

def plot_histogram(i: int, j: int):
    plt.subplot(height, width, i * height + j + 1)
    plt.plot(histograms[i, j])
    # plt.title(f"i={i}, j={j}")
    # plt.yscale("log")

    plt.xlabel("")
    plt.xticks([])
    plt.yticks([])
    plt.ylabel("")

    if ylim is not None:
        plt.ylim(None, ylim)
    if min_bin is not None:
        plt.xlim(min_bin, None)
    if max_bin is not None:
        plt.xlim(None, max_bin)

plt.figure(figsize=(height * 2, width * 2), dpi=400)
for i in range(height):
    for j in range(width):
        plot_histogram(i, j)
plt.tight_layout()

In [ ]:
PIXEL_SIZE = [16.8e-6, 38.8e-6]  # Pixel size in meters
FOCAL_LENGTH = 400e-6  # Focal length in meters

C = 3e8  # Speed of light in meters per second

CNH_SUBSAMPLE = 1
MIN_RESOLUTION = 250e-12
BIN_RESOLUTION = MIN_RESOLUTION * CNH_SUBSAMPLE
FOV_X = 45  # Field of view in x-direction [deg]
FOV_Y = 45  # Field of view in y-direction [deg]

def extract_point_cloud_interpolated(
    hists: np.ndarray, N: int, window: int = 5
) -> np.ndarray:
    fx = FOCAL_LENGTH / PIXEL_SIZE[0] * 3
    fy = FOCAL_LENGTH / PIXEL_SIZE[1] * 2
    cx = hists.shape[1] / 2 
    cy = hists.shape[0] / 2
    H, W, _ = hists.shape

    half = window // 2
    points = []
    for i in range(H):
        for j in range(W):
            hist = hists[i, j]
            idx = np.argmax(hist)
            start = max(0, idx - half)
            end = min(hist.size, idx + half + 1)
            bins = np.arange(start, end)
            w = hist[bins].astype(float)
            if w.sum() > 0:
                w /= w.sum()
            t_vals = bins * BIN_RESOLUTION / 2
            depth = (C * (w @ t_vals)) / 2

            for u in range(N):
                for v in range(N):
                    x_sub = j + (v + 0.5) / N
                    y_sub = i + (u + 0.5) / N
                    X = (x_sub - cx) * depth / fx
                    Y = (y_sub - cy) * depth / fy
                    Z = depth
                    points.append([X, Y, Z])
    return np.array(points)

In [ ]:
pt_cloud = extract_point_cloud_interpolated(sensor.accumulate(index=0).reshape(height, width, -1), 1)
print(pt_cloud.reshape(height, width, -1)[int(height // 2), int(width // 2)])

# Flatten the point cloud for Plotly
x = pt_cloud[..., 0].flatten()
y = pt_cloud[..., 1].flatten()
z = pt_cloud[..., 2].flatten()

fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z, mode='markers', marker=dict(color='blue', size=3)
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'  # or 'cube' or 'manual' with 'aspectratio'
    )
)

fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# constants
C = 299792458  # speed of light in m/s
BIN_RESOLUTION = 250e-12  # bin resolution in seconds
START_BIN = 8  # start bin for distance calculation

# precomputed trig tables from earlier
VL53L5_Zone_Pitch8x8 = np.array([
    59.00,64.00,67.50,70.00,70.00,67.50,64.00,59.00,
    64.00,70.00,72.90,74.90,74.90,72.90,70.00,64.00,
    67.50,72.90,77.40,80.50,80.50,77.40,72.90,67.50,
    70.00,74.90,80.50,85.75,85.75,80.50,74.90,70.00,
    70.00,74.90,80.50,85.75,85.75,80.50,74.90,70.00,
    67.50,72.90,77.40,80.50,80.50,77.40,72.90,67.50,
    64.00,70.00,72.90,74.90,74.90,72.90,70.00,64.00,
    59.00,64.00,67.50,70.00,70.00,67.50,64.00,59.00
]).reshape(8, 8)
VL53L5_Zone_Yaw8x8 = np.array([
    135.00,125.40,113.20, 98.13, 81.87, 66.80, 54.60, 45.00,
    144.60,135.00,120.96,101.31, 78.69, 59.04, 45.00, 35.40,
    156.80,149.04,135.00,108.45, 71.55, 45.00, 30.96, 23.20,
    171.87,168.69,161.55,135.00, 45.00, 18.45, 11.31,  8.13,
    188.13,191.31,198.45,225.00,315.00,341.55,348.69,351.87,
    203.20,210.96,225.00,251.55,288.45,315.00,329.04,336.80,
    203.20,225.00,239.04,258.69,281.31,300.96,315.00,324.60,
    225.00,234.60,246.80,261.87,278.13,293.20,305.40,315.00
]).reshape(8, 8)

SinOfPitch = np.sin(np.deg2rad(VL53L5_Zone_Pitch8x8))
CosOfPitch = np.cos(np.deg2rad(VL53L5_Zone_Pitch8x8))
SinOfYaw   = np.sin(np.deg2rad(VL53L5_Zone_Yaw8x8))
CosOfYaw   = np.cos(np.deg2rad(VL53L5_Zone_Yaw8x8))

def convert_dist2xyz_8x8(distance_mm, nb_target_detected, target_status):
    X = np.zeros((8, 8))
    Y = np.zeros((8, 8))
    Z = np.zeros((8, 8))
    valid = (
        (nb_target_detected > 0) &
        (distance_mm > 0) &
        np.isin(target_status, [5, 6, 9])
    )
    Hyp = np.zeros_like(distance_mm, dtype=float)
    Hyp[valid] = distance_mm[valid] / SinOfPitch[valid]
    X[valid] = CosOfYaw[valid] * CosOfPitch[valid] * Hyp[valid]
    Y[valid] = SinOfYaw[valid] * CosOfPitch[valid] * Hyp[valid]
    Z[valid] = distance_mm[valid]
    return X, Y, Z

# get histograms
histograms = sensor.accumulate(index=index).reshape(8, 8, -1)
H, W, B = histograms.shape

# allocate
distance_mm = np.zeros((H, W))
nb_target_detected = np.zeros((H, W), dtype=int)
target_status = np.full((H, W), 5, dtype=int)  # assume valid status

window = 5
half = window // 2

THRESHOLD = 10

# compute distance with weighted average
for i in range(H):
    for j in range(W):
        hist = histograms[i, j]
        idx = np.argmax(hist)
        start = max(0, idx - half)
        end = min(B, idx + half + 1)
        bins = np.arange(start, end) + START_BIN
        w = hist[start:end].astype(float)
        if w.sum() > THRESHOLD:
            w /= w.sum()
            t = bins * BIN_RESOLUTION
            distance_mm[i, j] = (C * (w @ t) / 2) * 1e3
            nb_target_detected[i, j] = 1

# convert to XYZ and plot
X, Y, Z = convert_dist2xyz_8x8(distance_mm, nb_target_detected, target_status)
fig = go.Figure(data=[go.Scatter3d(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    mode='markers', marker=dict(size=4)
)])
fig.update_layout(scene=dict(
    xaxis_title='X (mm)', yaxis_title='Y (mm)', zaxis_title='Z (mm)', aspectmode='data'
))
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# constants
C = 299792458             # speed of light (m/s)
BIN_RESOLUTION = 250e-12  # bin width (s)
START_BIN = 8            # bin offset
WINDOW = 5
HALF = WINDOW // 2
RES = 8
PER_PX = np.deg2rad(45) / RES
THRESHOLD = 20

def get_point_cloud(sensor):
    # get raw histograms
    hists = sensor.accumulate(index=index).reshape(RES, RES, -1)
    H, W, B = hists.shape

    # compute weighted-average distance (mm)
    distance_mm = np.zeros((H, W), dtype=float)
    for i in range(H):
        for j in range(W):
            hist = hists[i, j]
            idx = hist.argmax()
            start = max(0, idx - HALF)
            end   = min(B, idx + HALF + 1)
            bins  = np.arange(start, end) + START_BIN
            w     = hist[start:end].astype(float)
            if w.sum() > THRESHOLD:
                w /= w.sum()
                t_mean = w @ (bins * BIN_RESOLUTION)
                distance_mm[i, j] = (C * t_mean / 2) * 1e3

    # build point cloud buffer
    buf = np.empty((H, W, 3), dtype=np.float32)
    it = np.nditer(distance_mm, flags=['multi_index'])
    for d in it:
        i, j = it.multi_index
        e = max(0.0, d)
        x = e * np.cos(i * PER_PX - np.deg2rad(45)/2 - np.deg2rad(90)) / 1000
        y = e * np.sin(j * PER_PX - np.deg2rad(45)/2) / 1000
        z = e / 1000
        buf[i, j] = [x, y, z]

    return buf, 3*4  # buf and point_size

# usage
buf, point_size = get_point_cloud(sensor)
pts = buf.reshape(-1, 3)
fig = go.Figure(data=[go.Scatter3d(
    x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
    mode='markers', marker=dict(size=4)
)])
fig.update_layout(scene=dict(
    xaxis_title='X (m)',
    yaxis_title='Y (m)',
    zaxis_title='Z (m)',
    aspectmode='data'
))
fig.show()
